# Mục lục
- [Khám phá dữ liệu và Tiền Xử lý](#Khám-phá-dữ-liệu-và-Tiền-Xử-lý)
  - [Dữ liệu](#Dữ-liệu)
  - [Câu hỏi đặt ra](#Câu-hỏi-đặt-ra)
  - [Xử lý dữ liệu thiếu](#Xử-lý-dữ-liệu-thiếu)

# Khám phá dữ liệu và Tiền Xử lý

# (Exploring Data & PreProcessing Data)



Khái quát về dự án (Tiếng Anh): https://github.com/cstotodile/ds_final/blob/main/README.md

# Dữ liệu

Các món hàng _Thời trang nam_ trên shopee

Input là các thuộc tính của món hàng mà được biểu hiện ra trên giao diện của shopee.vn cho người dùng thấy, và thuộc tính thời gian của món hàng bao gồm: Thời gian sản phẩm được đăng, thời gian sản phẩm được lấy dữ liệu về máy, và thời gian sản phẩm được bày bán

Output là số lượng hàng đã bán được trong thời gian món hàng được đăng lên shopee cho đến lúc được lấy dữ liệu về máy

# Câu hỏi đặt ra

Dự đoán số lượng hàng bán được của một sản phẩm _Thời trang Nam_ tại một thời điểm trên shopee.vn



In [38]:
import re
import pandas as pd
import numpy as np

from numpy import mean
from numpy import std
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
# from sklearn.cross_validation import train_test_split

# Mở file và chỉnh sửa thông số hiển thị

Do dataset hiện đang nằm ở nhiều file khác nhau do có category khác nhau nên ta tiến hành gộp data bằng lệnh concat của pandas.

In [80]:
#Set Option
pd.set_option('max_colwidth', 10000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [123]:
# open

aoNganTayKhongCo_df = pd.read_csv('Data/ShoppeItem_thoiTrangNam_aoNganTayKhongCo_0-949.tsv', sep='\t')
aoSoMi_df = pd.read_csv('Data/ShoppeItem_thoiTrangNam_aoSoMi_0-949.tsv', sep='\t')
baloTuiVi_df = pd.read_csv('Data/ShoppeItem_thoiTrangNam_baloTuiVi_0-949.tsv', sep='\t')
giayDepNam_df = pd.read_csv('Data/ShoppeItem_thoiTrangNam_giayDepNam_0-949.tsv', sep='\t')
phuKienNam_df = pd.read_csv('Data/ShoppeItem_thoiTrangNam_phuKienNam_0-949.tsv', sep='\t')
Quan_df = pd.read_csv('Data/ShoppeItem_thoiTrangNam_Quan_0-949.tsv', sep='\t')

frames = [aoNganTayKhongCo_df,aoSoMi_df,baloTuiVi_df,giayDepNam_df,phuKienNam_df,Quan_df]

items_df = pd.concat(frames, ignore_index=True, sort=False)

# Sử dụng lệnh dưới thay vì lệnh đang dùng ở phía trên, các dataframe được gộp với nhau 
# nhưng vẫn có thể gọi riêng một dataframe thông qua key.
# Tuy nhiên chỉ số index sẽ không chạy từ 0 đến hết mà giữ nguyên ở mỗi bảng.

# items_df = pd.concat(frames, keys = [aoNganTayKhongCo', 'aoSoMi', 'baloTuiVi', 'giayDepNam', 'phuKienNam', 'Quan'])


In [124]:
items_df

,name,shop_location,item_public_time,item_be_got_time,sell_time,shopee_verified,discount,is_adult,options,show_official_shop_label_in_title,rating_star,five_star,four_star,three_star,two_star,one_star,flash_sale,upcoming_flash_sale,price_min,price_min_before_discoun,price_max,price_max_before_discount,price,price_before_discount,coin_earn_label,liked_count,view_count,cmt_count,is_preferred_plus_seller,show_free_shipping,images,video_info_list,historical_sold
0,Áo thun phông Unisex nam nữ 2 màu Trắng Đen tay ngắn cổ tròn dáng ôm phối cực cá tính - V107,TP. Hồ Chí Minh,1.607317e+09,1.609949e+09,2632455.0,True,39%,False,8.0,False,5.000000,4.0,0.0,0.0,0.0,0.0,None,None,1.490000e+10,2.450000e+10,1.490000e+10,2.450000e+10,1.490000e+10,2.450000e+10,None,7.0,1048.0,4.0,True,False,8.0,0.0,18.0
1,Áo Thun Trơn Nam [FREESHIP] Áo phông nam tay ngắn 100% cotton Trắng Đen dáng ôm vừa vặn AT01,TP. Hồ Chí Minh,1.596338e+09,1.609949e+09,13611374.0,True,49%,False,35.0,False,4.947977,166.0,5.0,2.0,0.0,0.0,None,None,1.090000e+10,1.090000e+10,1.090000e+10,2.150000e+10,1.090000e+10,1.090000e+10,None,109.0,10540.0,173.0,True,False,9.0,0.0,526.0
2,"[SALE 50%] - Áo Thun Nam Cổ Tròn Cao Cấp ( nhiều màu) Tay Ngắn, chất cotton bề mặt vải mềm Mịn cực hot",TP. Hồ Chí Minh,1.603988e+09,1.609949e+09,5961754.0,True,50%,False,48.0,False,4.885417,89.0,4.0,2.0,1.0,0.0,None,None,4.950000e+09,9.900000e+09,4.950000e+09,9.900000e+09,4.950000e+09,9.900000e+09,None,25.0,4025.0,96.0,True,False,9.0,0.0,431.0
3,Áo thun nam unisex 100% cotton basic Trắng Đen áo phông co giãn cao cấp dáng rộng dễ phối đồ - V69,TP. Hồ Chí Minh,1.596083e+09,1.609949e+09,13866721.0,True,39%,False,8.0,False,5.000000,66.0,0.0,0.0,0.0,0.0,None,None,1.490000e+10,2.450000e+10,1.490000e+10,2.450000e+10,1.490000e+10,2.450000e+10,None,85.0,4551.0,66.0,True,False,7.0,0.0,141.0
4,🔥XẢ HÀNG🔥 Áo đôi - áo cặp nam nữ dễ thương cực HOT,Hà Nội,1.590391e+09,1.609949e+09,19558163.0,False,None,False,10.0,False,4.687500,27.0,1.0,3.0,1.0,0.0,None,None,7.200000e+09,-1.000000e+00,7.200000e+09,-1.000000e+00,7.200000e+09,0.000000e+00,None,128.0,3613.0,32.0,False,False,1.0,0.0,158.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5746,"Quần thời trang nam, quần thể thao nam Mã One010 One shop",Hà Nội,1.581356e+09,1.609992e+09,28635448.0,True,47%,False,18.0,False,4.857143,6.0,1.0,0.0,0.0,0.0,None,None,6.900000e+09,1.290000e+10,6.900000e+09,1.290000e+10,6.900000e+09,1.290000e+10,None,4.0,35.0,7.0,False,True,8.0,0.0,14.0
5747,"Quần đùi nam họa tiết đẹp mắt thời trang nam phom 40-75kg, Mã KQ40 kynn shop",Hà Nội,1.597771e+09,1.609992e+09,12220879.0,True,47%,False,6.0,False,5.000000,2.0,0.0,0.0,0.0,0.0,None,None,6.900000e+09,1.290000e+10,6.900000e+09,1.290000e+10,6.900000e+09,1.290000e+10,None,0.0,40.0,2.0,True,False,9.0,0.0,4.0
5748,Quần lót lọt khe lưới xuyên thấu thời trang nam gợi cảm,Nước ngoài,1.570801e+09,1.609992e+09,39190550.0,False,48%,False,0.0,False,5.000000,1.0,0.0,0.0,0.0,0.0,None,None,4.742400e+09,9.120000e+09,4.742400e+09,9.120000e+09,4.742400e+09,9.120000e+09,None,4.0,12.0,1.0,False,True,9.0,0.0,2.0
5749,[FREESHIP_99K] Bộ 5 quần lót Boxer thời trang nam - Thun cotton cao cấp - VV07,TP. Hồ Chí Minh,1.552326e+09,1.609992e+09,57665762.0,True,22%,False,3.0,False,4.833333,22.0,1.0,0.0,1.0,0.0,None,None,8.500000e+09,1.090000e+10,8.500000e+09,1.090000e+10,8.500000e+09,1.090000e+10,None,4965.0,13.0,24.0,False,True,6.0,0.0,49.0


Thử kiểm tra lại thông số về số mẫu và số thuộc tính của dataset

In [125]:
items_df.shape

(5751, 33)

In [126]:
items_df.columns.values

array(['name', 'shop_location', 'item_public_time', 'item_be_got_time',
       'sell_time', 'shopee_verified', 'discount', 'is_adult', 'options',
       'show_official_shop_label_in_title', 'rating_star', 'five_star',
       'four_star', 'three_star', 'two_star', 'one_star', 'flash_sale',
       'upcoming_flash_sale', 'price_min', 'price_min_before_discoun',
       'price_max', 'price_max_before_discount', 'price',
       'price_before_discount', 'coin_earn_label', 'liked_count',
       'view_count', 'cmt_count', 'is_preferred_plus_seller',
       'show_free_shipping', 'images', 'video_info_list',
       'historical_sold'], dtype=object)

Sau khi kiểm tra, ta tiến hành lưu lại dữ liệu đã gộp

In [102]:
items_df.to_csv('Data/ShoppeItem_full.tsv', sep = '\t', index=False)

# Xử lý dữ liệu thiếu

Do dữ liệu trả về từ api của shopee những dữ liệu thiếu có dạng string: 'None' nên pandas sẽ không đếm được lượng dữ liệu thiếu này. Ta tiến hành thay 'None' thành pd.NA để có thể đếm dữ liệu thiếu chính xác.

In [127]:
# cond = items_df != 'None'
# items_df.where(cond, pd.NA)
items_df = items_df.replace('None', pd.NA)

In [128]:
items_df.describe()

,item_public_time,item_be_got_time,sell_time,options,rating_star,five_star,four_star,three_star,two_star,one_star,price_min,price_min_before_discoun,price_max,price_max_before_discount,price,price_before_discount,liked_count,view_count,cmt_count,images,video_info_list,historical_sold
count,5.750000e+03,5.750000e+03,5.750000e+03,5750.000000,5750.000000,5750.000000,5750.000000,5750.000000,5750.000000,5750.000000,5.750000e+03,5.750000e+03,5.750000e+03,5.750000e+03,5.750000e+03,5.750000e+03,5750.000000,5750.000000,5750.000000,5750.000000,5750.000000,5750.000000
mean,1.588962e+09,1.609952e+09,2.098975e+07,11.130435,1.480957,22.178783,1.454261,0.640348,0.217391,0.425739,2.497484e+10,1.618696e+10,2.570559e+10,1.659865e+10,2.497484e+10,1.618696e+10,408.308174,400.182261,24.958783,6.486261,0.102087,130.039478
std,2.241358e+07,3.237006e+04,2.241150e+07,11.856363,2.224479,223.378303,18.396975,9.413689,3.849496,8.287135,3.017264e+10,2.351174e+10,3.048769e+10,2.402012e+10,3.017264e+10,2.351174e+10,2415.384539,3868.961458,260.201111,2.786682,0.302789,1105.836068
min,1.455984e+09,1.609910e+09,5.430400e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.900000e+08,-1.000000e+00,3.900000e+08,-1.000000e+00,3.900000e+08,0.000000e+00,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,1.581753e+09,1.609917e+09,4.393969e+06,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.900000e+09,-1.000000e+00,9.900000e+09,-1.000000e+00,9.900000e+09,0.000000e+00,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000
50%,1.597173e+09,1.609949e+09,1.277768e+07,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.669000e+10,6.000000e+09,1.700000e+10,6.590000e+09,1.669000e+10,6.000000e+09,0.000000,0.000000,0.000000,8.000000,0.000000,0.000000
75%,1.605586e+09,1.609992e+09,2.820339e+07,15.000000,4.699226,1.000000,0.000000,0.000000,0.000000,0.000000,2.841750e+10,2.500000e+10,2.990000e+10,2.500000e+10,2.841750e+10,2.500000e+10,7.000000,28.000000,1.000000,9.000000,0.000000,3.000000
max,1.609856e+09,1.609994e+09,1.539267e+08,80.000000,5.000000,7869.000000,777.000000,427.000000,199.000000,462.000000,6.900000e+11,2.680000e+11,6.900000e+11,2.680000e+11,6.900000e+11,2.680000e+11,79919.000000,189252.000000,9737.000000,9.000000,1.000000,37079.000000


In [129]:
items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5751 entries, 0 to 5750
Data columns (total 33 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   name                               5751 non-null   object 
 1   shop_location                      5694 non-null   object 
 2   item_public_time                   5750 non-null   float64
 3   item_be_got_time                   5750 non-null   float64
 4   sell_time                          5750 non-null   float64
 5   shopee_verified                    5750 non-null   object 
 6   discount                           3030 non-null   object 
 7   is_adult                           5749 non-null   object 
 8   options                            5750 non-null   float64
 9   show_official_shop_label_in_title  5685 non-null   object 
 10  rating_star                        5750 non-null   float64
 11  five_star                          5750 non-null   float

Nhận xét:

1. Ta thấy 3 thuộc tính flash_sale, upcoming_flash_sale, coin_earn_label đều bị thiếu dữ liệu (0 non-null) nên ta tiến hành xóa bỏ ba thuộc tính này khỏi dataset.

2. Ta lại thấy có một mẫu bị thiếu tất cả dữ liệu trừ thuộc tính name, tiến hành xóa mẫu này.

Để đơn giản, ta có thể dùng phương thức dropna của pandas để xóa các dòng và cột bị thiếu toàn bộ dữ liệu:

1. Truyền tham số how = 'all' và axis = 1  cho cột

2. Truyền tham số thresh = 15 (những mẫu thiếu từ 15 thuộc tính) và axis = 0 cho dòng.

In [131]:
items_df = items_df.dropna(axis = 0, thresh = 15)
items_df = items_df.dropna(axis = 1, how = 'all')
# items_df = items_df.drop('flash_sale', axis = 1)
# items_df = items_df.drop('upcoming_flash_sale', axis = 1)
# items_df = items_df.drop('coin_earn_label', axis = 1)

In [132]:
items_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5750 entries, 0 to 5750
Data columns (total 30 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   name                               5750 non-null   object 
 1   shop_location                      5694 non-null   object 
 2   item_public_time                   5750 non-null   float64
 3   item_be_got_time                   5750 non-null   float64
 4   sell_time                          5750 non-null   float64
 5   shopee_verified                    5750 non-null   object 
 6   discount                           3030 non-null   object 
 7   is_adult                           5749 non-null   object 
 8   options                            5750 non-null   float64
 9   show_official_shop_label_in_title  5685 non-null   object 
 10  rating_star                        5750 non-null   float64
 11  five_star                          5750 non-null   float

Kiểm tra lại ta thấy thuộc tính discount gần như có một nửa giá trị bị thiếu. Kiểm tra đối chiếu với các item trên shopee, ta thấy thực tế những mẫu bị thiếu dữ liệu ở thuộc tính discount là do không có giảm giá (discount = 0). Ta tiến hành sửa bằng cách thay giá trị NA tại cột này thành 0.

In [133]:
items_df['discount'] = items_df['discount'].fillna(0)

In [134]:
items_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5750 entries, 0 to 5750
Data columns (total 30 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   name                               5750 non-null   object 
 1   shop_location                      5694 non-null   object 
 2   item_public_time                   5750 non-null   float64
 3   item_be_got_time                   5750 non-null   float64
 4   sell_time                          5750 non-null   float64
 5   shopee_verified                    5750 non-null   object 
 6   discount                           5750 non-null   object 
 7   is_adult                           5749 non-null   object 
 8   options                            5750 non-null   float64
 9   show_official_shop_label_in_title  5685 non-null   object 
 10  rating_star                        5750 non-null   float64
 11  five_star                          5750 non-null   float

Đến đây ta thấy: 

1. Cột shop_location, is_adult có một vài mẫu bị thiếu dữ liệu. 
  - Do số lượng mẫu thiếu dữ liệu ít 
  - Thuộc tính shop_location 
  - -> Vậy nên ta có thể xóa những mẫu này.


2. Thuộc tính show_official_shop_label_in_title thiếu khá nhiều thuộc tính. Tuy nhiên: 
  - Thứ ta quan tâm là giao diện người dùng thấy gì trên shopee và những mẫu này vẫn đang hoạt động tốt trên shopee
  - Thuộc tính show_official_shop_label_in_title có dạng bool, có thể xảy ra trường hợp cột này vô nghĩa vì toàn bộ false hoặc toàn bộ true.
  -> Vậy nên ta sẽ kiểm tra sau mà chưa vội điền dữ liệu thiếu (impute)

In [142]:
items_df = items_df.dropna(subset = ['shop_location', 'is_adult'])

In [143]:
items_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5693 entries, 0 to 5750
Data columns (total 30 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   name                               5693 non-null   object 
 1   shop_location                      5693 non-null   object 
 2   item_public_time                   5693 non-null   float64
 3   item_be_got_time                   5693 non-null   float64
 4   sell_time                          5693 non-null   float64
 5   shopee_verified                    5693 non-null   object 
 6   discount                           5693 non-null   object 
 7   is_adult                           5693 non-null   object 
 8   options                            5693 non-null   float64
 9   show_official_shop_label_in_title  5672 non-null   object 
 10  rating_star                        5693 non-null   float64
 11  five_star                          5693 non-null   float

Đến đây ta đã không còn giá trị thiếu, tuy nhiên có thể có giá trị không hợp lệ. Thật ra giá trị của những mẫu này đều đang hoạt động bình thường trên giao diện shopee nên khả năng lỗi là rất thấp. Ta vẫn kiểm tra lại như sau:
1. Có giá trị âm ở các cột dạng numeric không
2. So sánh price_min và price_max, price_min_before_discount và price_max_before_discount
3. So sánh price và price_before_discount, price_min và price_min_before_discount, price_max và price_max_before_discount

In [14]:
items_df['shop_location'].unique()

array(['TP. Hồ Chí Minh', 'Hà Nội', 'Nước ngoài', 'Bà Rịa - Vũng Tàu',
       'Phú Thọ', 'Bắc Ninh', 'Quảng Ninh', 'Lâm Đồng', 'None',
       'Hưng Yên', 'Hải Dương', 'Thái Bình'], dtype=object)

In [20]:
shop_locations = ['TP. Hồ Chí Minh', 'Hà Nội', 'Nước ngoài', 'Bà Rịa - Vũng Tàu',
       'Phú Thọ', 'Bắc Ninh', 'Quảng Ninh', 'Lâm Đồng', 'None',
       'Hưng Yên', 'Hải Dương', 'Thái Bình']

In [24]:
X = items_df['shop_location']
X = X[:, np.newaxis]
X

array([['TP. Hồ Chí Minh'],
       ['TP. Hồ Chí Minh'],
       ['TP. Hồ Chí Minh'],
       ['TP. Hồ Chí Minh'],
       ['TP. Hồ Chí Minh'],
       ['TP. Hồ Chí Minh'],
       ['Hà Nội'],
       ['TP. Hồ Chí Minh'],
       ['Hà Nội'],
       ['Hà Nội'],
       ['Hà Nội'],
       ['TP. Hồ Chí Minh'],
       ['Hà Nội'],
       ['TP. Hồ Chí Minh'],
       ['Nước ngoài'],
       ['Hà Nội'],
       ['Hà Nội'],
       ['Nước ngoài'],
       ['Hà Nội'],
       ['Hà Nội'],
       ['TP. Hồ Chí Minh'],
       ['Hà Nội'],
       ['Hà Nội'],
       ['Hà Nội'],
       ['Hà Nội'],
       ['Nước ngoài'],
       ['Hà Nội'],
       ['Hà Nội'],
       ['TP. Hồ Chí Minh'],
       ['Nước ngoài'],
       ['TP. Hồ Chí Minh'],
       ['Hà Nội'],
       ['Hà Nội'],
       ['Hà Nội'],
       ['Hà Nội'],
       ['Hà Nội'],
       ['Hà Nội'],
       ['Bà Rịa - Vũng Tàu'],
       ['TP. Hồ Chí Minh'],
       ['Nước ngoài'],
       ['Nước ngoài'],
       ['TP. Hồ Chí Minh'],
       ['Hà Nội'],
       ['Nước ngoài'],

In [30]:
enc = OneHotEncoder(categories=[shop_locations])
enc.fit(X)

OneHotEncoder(categories=[['TP. Hồ Chí Minh', 'Hà Nội', 'Nước ngoài',
                           'Bà Rịa - Vũng Tàu', 'Phú Thọ', 'Bắc Ninh',
                           'Quảng Ninh', 'Lâm Đồng', 'None', 'Hưng Yên',
                           'Hải Dương', 'Thái Bình']])

In [37]:
X_ = enc.transform(X).toarray()
print(X[351])
print(X_[351])

['Nước ngoài']
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [17]:
items_df['name'].describe()

count                                    950
unique                                   648
top       Áo Khoác Sơ Mi Caro Thời Trang Nam
freq                                     140
Name: name, dtype: object

In [40]:
simple_imp= SimpleImputer()
imp_mode = SimpleImputer(strategy='most_frequent')

#shop_location
knn_imp = KNNImputer(n_neighbors = 3)

In [ ]:
imp_ss_pipe = make_pipeline(simple_imp, StandardScaler())

In [ ]:
steps = list()
steps.append(('scaler', MinMaxScaler()))
pipeline = Pipeline(steps=steps)

In [28]:
type(items_df['rating_count'][0])

str

In [22]:
num_duplicated_rows = items_df.index.duplicated(keep='first')
num_duplicated_rows = np.count_nonzero(num_duplicated_rows)
num_duplicated_rows

0

In [27]:
dtypes = items_df.dtypes
dtypes

name                                  object
shop_location                         object
shopee_verified                         bool
discount                              object
is_adult                                bool
options                                int64
show_official_shop_label_in_title       bool
rating_star                          float64
rating_count                          object
flash_sale                            object
upcoming_flash_sale                   object
price_min                              int64
price_min_before_discoun               int64
price_max                              int64
price_max_before_discount              int64
price                                  int64
price_before_discount                  int64
coin_earn_label                       object
liked_count                            int64
view_count                             int64
cmt_count                              int64
is_preferred_plus_seller                bool
show_free_